<a href="https://colab.research.google.com/github/c-marq/cap4767-data-mining/blob/main/solutions/exercises/week05_group_exercise_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 5 Group Exercise — SOLUTION KEY 🔑 — RFM + K-Means on a New Dataset
**CAP4767 Data Mining with Python** | Miami Dade College — Kendall Campus

**Points:** 10 | **Duration:** ~45 minutes | **Deliverable:** Completed notebook + 3-minute presentation

**Objective:** Run the complete RFM + K-Means pipeline on a **different dataset** from the demo. Apply what you learned in both sessions — RFM scoring, CLTV, manual segmentation, K-Means clustering, and segment comparison.

### Group Members & Roles

| Role | Name | Responsibility |
|------|------|----------------|
| 🖥️ **Lead Coder** | | Drives the notebook |
| 📊 **Data Interpreter** | | Reads outputs, explains what the numbers mean |
| 🎤 **Presenter** | | Delivers the 3-minute share-out |
| ✅ **QA Reviewer** | | Checks outputs against checkpoints |

*If 3 members, Lead Coder also handles QA.*

<div style="background-color: #D6EAF8; border-left: 5px solid #2E86C1; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1A5276;">💡 GROUP DISCUSSION (before coding — 3 minutes)</strong><br>
  Your dataset tracks a different type of customer than the UK online retailer from the demo. Before running any code:
  <ol>
    <li>What does <strong>Recency</strong> mean in this context? What column represents it, or how will you calculate it?</li>
    <li>What does <strong>Frequency</strong> mean? Is it the number of orders, visits, or something else?</li>
    <li>What does <strong>Monetary</strong> mean? Is it revenue, volume, or a different measure?</li>
    <li>Do you expect the Pareto concentration to be more or less extreme than the Online Retail II dataset? Why?</li>
  </ol>
</div>

**Our group's answers (2–3 sentences per question):**

**Sample (Restaurant):** (1) Recency = days since this order's date to the snapshot date. Since each order is treated as a unique customer, Recency reflects when that order occurred. (2) Frequency will be 1 for all orders since each order number is unique — this is a limitation of the dataset. (3) Monetary = total order value (Quantity × Product Price summed across all items in the order). (4) We expect less extreme Pareto concentration because restaurant orders tend to have a narrower price range than online retail — you can't easily spend £10,000 on curry like you can on wholesale goods.

---

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Run the setup cell. Then <strong>uncomment ONE dataset option</strong> in the data loading cell and run it.
</div>

In [ ]:
# ============================================================
# Setup — Run this cell. Do not modify.
# ============================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

np.random.seed(42)
plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["figure.dpi"] = 100
sns.set_style("whitegrid")

# Segmentation function from the demo
def segment_customer(score):
    if score >= 12:
        return "Champions"
    elif score >= 9:
        return "Loyal"
    elif score >= 7:
        return "Potential Loyalist"
    elif score >= 5:
        return "At Risk"
    elif score >= 4:
        return "Can't Lose"
    else:
        return "Lost"

print("✅ Setup complete — segment_customer() function loaded")

In [ ]:
# ============================================================
# OPTION A — Brazilian E-Commerce (Olist) — ~100K orders
# More challenging: requires merging 3 tables
# Uncomment this entire block if choosing Option A
# ============================================================
# base = "https://raw.githubusercontent.com/c-marq/cap4767-data-mining/refs/heads/main/data"
# customers = pd.read_csv(f"{base}/olist_customers_dataset.csv")
# orders = pd.read_csv(f"{base}/olist_orders_dataset.csv")
# payments = pd.read_csv(f"{base}/olist_order_payments_dataset.csv")
#
# # Merge: orders → customers (to get customer_unique_id) → payments (to get revenue)
# merged = orders.merge(customers, on="customer_id").merge(payments, on="order_id")
#
# # Use customer_unique_id (not customer_id — one person can have multiple customer_ids)
# df = merged[["customer_unique_id", "order_purchase_timestamp", "order_id", "payment_value"]].copy()
# df.columns = ["CustomerID", "InvoiceDate", "InvoiceNo", "TotalPrice"]
# df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
# df = df.dropna(subset=["CustomerID"])
# df = df[df["TotalPrice"] > 0]
# DOMAIN = "Brazilian E-Commerce"

# ============================================================
# OPTION B — Restaurant Orders — ~1K orders
# Simpler, beginner-friendly
# Uncomment this entire block if choosing Option B
# ============================================================
url = "https://raw.githubusercontent.com/c-marq/cap4767-data-mining/refs/heads/main/data/restaurant-1-orders.csv"
df_raw = pd.read_csv(url)

# Restaurant data doesn't have CustomerID — use Order Number as proxy
# Each order is one "visit" — Frequency = number of visits
df = df_raw.copy()
df["TotalPrice"] = df["Quantity"] * df["Product Price"]
df["InvoiceDate"] = pd.to_datetime(df["Order Date"], format="%d/%m/%Y %H:%M")
df.rename(columns={"Order Number": "InvoiceNo"}, inplace=True)

# Create a CustomerID proxy: since we don't have one, group by Order Number
# For this dataset, each order IS a customer visit — we'll aggregate at order level first
# then treat each unique order as a "customer" for RFM
# NOTE: This is a simplification — discuss with your group why this is a limitation
order_summary = df.groupby("InvoiceNo").agg(
    InvoiceDate=("InvoiceDate", "first"),
    TotalPrice=("TotalPrice", "sum")
).reset_index()
order_summary["CustomerID"] = order_summary["InvoiceNo"]  # Each order = unique customer
df = order_summary
DOMAIN = "Restaurant Orders"

print(f"Dataset: {DOMAIN}")
print(f"Shape: {df.shape[0]:,} rows")
print(f"Unique customers/orders: {df['CustomerID'].nunique():,}")
print(f"Date range: {df['InvoiceDate'].min()} to {df['InvoiceDate'].max()}")
df.head()

<div style="background-color: #FEF9E7; border-left: 5px solid #F1C40F; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #7D6608;">⚠️ DATASET NOTES</strong><br>
  <ul>
    <li><strong>Option A (Olist):</strong> Uses <code>customer_unique_id</code> (not <code>customer_id</code>) because one person can have multiple IDs across orders. Revenue = <code>payment_value</code>.</li>
    <li><strong>Option B (Restaurant):</strong> No true CustomerID — each order number is treated as a unique customer. This limits RFM interpretation (Frequency will always be 1). Discuss this limitation in your share-out.</li>
  </ul>
</div>

---
## Task 1 — Calculate RFM Metrics (1 code cell)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Set a snapshot date (1 day after the last transaction). Use <code>groupby</code> on CustomerID to calculate Recency, Frequency, and Monetary. Print <code>rfm.describe()</code>.
</div>

In [ ]:
# Task 1: Calculate RFM metrics
snapshot_date = df["InvoiceDate"].max() + pd.Timedelta(days=1)

rfm = df.groupby("CustomerID").agg(
    Recency=("InvoiceDate", lambda x: (snapshot_date - x.max()).days),
    Frequency=("InvoiceNo" if "InvoiceNo" in df.columns else "CustomerID", "nunique"),
    Monetary=("TotalPrice", "sum")
).reset_index()

print(f"Unique customers: {rfm.shape[0]:,}")
print(f"Snapshot date: {snapshot_date}")
print(f"\nRFM Summary:")
print(rfm[["Recency", "Frequency", "Monetary"]].describe().round(2))

<div style="background-color: #FADBD8; border-left: 5px solid #E74C3C; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #922B21;">🛑 CHECKPOINT 1</strong><br>
  Your <code>rfm</code> DataFrame should have one row per customer and three numeric columns. Recency should be in days, Frequency should use <code>nunique</code>, Monetary should be a sum.
</div>

---
## Task 2 — Assign RFM Scores and Manual Segments (1 code cell)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Apply <code>pd.qcut()</code> for R, F, M quintile scores (1–5). Use <code>.rank(method='first')</code> on Frequency if you get a bin-edge error. Calculate composite score. Apply <code>segment_customer()</code>. Print value_counts.
</div>

In [ ]:
# Task 2: RFM scores + manual segments
rfm["R_Score"] = pd.qcut(rfm["Recency"], q=5, labels=[5, 4, 3, 2, 1]).astype(int)
rfm["F_Score"] = pd.qcut(rfm["Frequency"].rank(method="first"), q=5, labels=[1, 2, 3, 4, 5]).astype(int)
rfm["M_Score"] = pd.qcut(rfm["Monetary"], q=5, labels=[1, 2, 3, 4, 5]).astype(int)
rfm["RFM_Score"] = rfm["R_Score"] + rfm["F_Score"] + rfm["M_Score"]

rfm["Customer_Segment"] = rfm["RFM_Score"].apply(segment_customer)

print("Customer Segments:")
print(rfm["Customer_Segment"].value_counts())

**Interpretation:** Which segment is largest? Which has the fewest customers? Does this distribution surprise you?

**Sample:** Potential Loyalist is the largest segment, which makes sense — most restaurant orders fall in the mid-range for both price and recency. Champions is relatively small because truly high-spending orders are rare at a restaurant. The Lost segment is also small since the dataset covers a narrow time window.

<div style="background-color: #FADBD8; border-left: 5px solid #E74C3C; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #922B21;">🛑 CHECKPOINT 2</strong><br>
  You should have 6 segment labels. If all customers land in one segment, check your qcut labels — especially the Recency reversal.
</div>

---
## Task 3 — Scale Features and Run Elbow Method (1 code cell)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Use <code>StandardScaler</code> on R, F, M scores. Run K-Means for k=2 through k=8. Plot the inertia curve.
</div>

In [ ]:
# Task 3: Scale + Elbow Method
features = rfm[["R_Score", "F_Score", "M_Score"]]
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

inertias = []
k_range = range(2, 9)
for k in k_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(features_scaled)
    inertias.append(km.inertia_)

plt.figure(figsize=(10, 5))
plt.plot(k_range, inertias, "bo-", linewidth=2, markersize=8)
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Inertia")
plt.title("Elbow Method")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

**Chosen k and justification (1–2 sentences):**

**Sample:** The elbow appears at k=4 — the drop from k=3 to k=4 is substantial, then the curve flattens. We chose k=4 because adding a 5th cluster doesn't meaningfully reduce inertia.

<div style="background-color: #FADBD8; border-left: 5px solid #E74C3C; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #922B21;">🛑 CHECKPOINT 3</strong><br>
  Your elbow plot should show a clear curve. If it's a straight line, check that you're using the scaled features, not the raw values.
</div>

---
## Task 4 — Fit K-Means and Profile Clusters (1 code cell)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Fit K-Means with your chosen k. Add cluster labels to <code>rfm</code>. Create a cluster profiling heatmap (mean R, F, M per cluster). Print cluster sizes.
</div>

In [ ]:
# Task 4: K-Means + profiling heatmap
optimal_k = 4  # Adjust based on your elbow plot
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
rfm["Cluster"] = kmeans.fit_predict(features_scaled)

print(f"Cluster sizes (k={optimal_k}):")
print(rfm["Cluster"].value_counts().sort_index())

# Profiling heatmap
cluster_profile = rfm.groupby("Cluster")[["R_Score", "F_Score", "M_Score"]].mean()
plt.figure(figsize=(8, 5))
sns.heatmap(cluster_profile, annot=True, fmt=".2f", cmap="YlOrRd",
            xticklabels=["Recency", "Frequency", "Monetary"], linewidths=0.5)
plt.title("Cluster Profiles — Mean RFM Scores")
plt.ylabel("Cluster")
plt.tight_layout()
plt.show()

**Name each cluster (1 sentence per cluster):**

**Sample:** Cluster 0: 'Big Spenders' — high M, moderate R and F. Cluster 1: 'Recent Light Buyers' — high R, low M. Cluster 2: 'Lapsed Customers' — low R, low F, low M. Cluster 3: 'Core Regulars' — moderate across all three dimensions.

---
## Task 5 — Compare Manual Segments vs K-Means (1 code cell)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Build a crosstab: <code>pd.crosstab(rfm["Customer_Segment"], rfm["Cluster"], margins=True)</code>
</div>

In [ ]:
# Task 5: Crosstab comparison
crosstab = pd.crosstab(rfm["Customer_Segment"], rfm["Cluster"], margins=True)
print("Manual Segments (rows) vs K-Means Clusters (columns):")
print(crosstab)

**Interpretation (2–3 sentences):** Where do the two approaches agree? Where do they disagree? Did K-Means discover a sub-group within one of the manual segments?

**Sample:** The two methods agree that the highest-scoring customers (Champions) cluster together. They disagree on the mid-range — the 'At Risk' manual segment is split across two clusters, one with higher Monetary and one with lower. K-Means separated 'At Risk customers who spent a lot but haven't returned' from 'At Risk customers who barely spent and haven't returned' — the manual rules lumped them together.

---
## Six Success Criteria Questions

Answer each in 1–3 sentences:

1. **How many unique customers are in your dataset after cleaning?**
2. **What percentage of customers fall into the top two RFM segments (Champions + Loyal)?**
3. **What is the ratio of mean to median Monetary value?** (Revenue concentration indicator)
4. **How many clusters did the Elbow Method suggest, and do you agree?**
5. **Which K-Means cluster represents the highest-value customers, and how many are in it?**
6. **Did K-Means reveal a group that the manual rules missed or split?** Describe it.

**Sample answers:**
1. ~660 unique orders after cleaning.
2. Champions + Loyal ≈ 35–40% of the base.
3. Mean/Median Monetary ratio ≈ 1.4x — less extreme than Online Retail because restaurant orders have a narrower price range.
4. Elbow suggests k=4, and we agree — the curve clearly flattens after 4.
5. Cluster 0 (Big Spenders) — approximately 150 customers with the highest average Monetary scores.
6. K-Means split the At Risk segment into two distinct groups: one with high historical spend (worth a win-back campaign) and one with minimal spend (lower priority). The manual rules treated them identically.

---

## Share-Out (3 minutes)

Present to the class:
1. Which dataset your group used and how you mapped R, F, M
2. Your most interesting finding from manual segmentation
3. Your most interesting finding from K-Means
4. One recommendation a business could act on based on your segmentation

## Troubleshooting

| Problem | Fix |
|---------|-----|
| `pd.qcut` error: "Bin edges must be unique" | Add `.rank(method='first')` before qcut |
| All customers in one segment | Check Recency label reversal: `labels=[5,4,3,2,1]` |
| Olist merge gives empty DataFrame | Check column names: `customer_id` (not `CustomerID`) in merge |
| Elbow plot is a straight line | Use scaled features, not raw scores |
| Restaurant Frequency is all 1 | Expected — each order is a unique "customer." Discuss as a limitation. |

---
<p style="color:#7F8C8D; font-size:0.85em;">
<em>CAP4767 Data Mining with Python | Miami Dade College | Spring 2026</em><br>
Week 5 Group Exercise — RFM + K-Means on a New Dataset | 10 Points
</p>